# RNA sequencing workflow example

In this notebook, we'll walk through an RNA sequencing workflow in Ovation for Service Labs. Although the activities in the workflow can be accomplished using the web app (https://lab.ovation.io) to download & upload files, this notebook illustrates the API interactions to complete the workflow using existing bioinformatics tools. 

## Setup

In [2]:
import urllib
import texttable
import os
import glob

import ovation.lab.workflows as workflows
# import ovation.lab.download as download

from ovation.session import connect

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [3]:
cwd = os.getcwd()

## Connection

This interactive notebooks starts with an interactive `Session` connection. If you already have a (long-lived) API token, you can create a session with:

    s = ovation.session.Session(token, api='https://services.ovation.io', token='/api/v1/sessions')

In [4]:
s = connect(input('Email: '), api='https://services-staging.ovation.io', token='/api/v1/sessions')

Email: barry@ovation.io
Ovation password: ········


## Workflow

We'll need to know which workflow to post data to.

In [5]:
workflow_id = input('Workflow ID: ')

Workflow ID: 132


In [6]:
r = s.get(s.entity_path('workflows', workflow_id))
workflow = r.workflow

Here's the full workflow: 
![title](workflow.png)

The burnt-orange activities are most easily accomplished in the web app, so we'll assume that they're completed in the app. The secions below show the API calls for the light-orange colored activities.

What samples are in the pool?

In [7]:
samples = s.get(workflow.links.samples)

table = texttable.Texttable()
table.set_deco(texttable.Texttable.HEADER)
# table.set_cols_align(["l", "r", "r", "r", "l"])
table.add_rows([["Identifier", "Date received"]] + [[s.identifier, s.date_received] for s in samples])
print(table.draw())

Identifier      Date received    
Lib-Sample   2016-09-06 00:00:00 


### Downloading files

In many activities, you'll want to download files from previous activities (e.g. the `fastq` files from demultiplexing in the Sequencing QC Prep activities). You can use `ovation.lab.download.download_resources` to get the resources from a labeled activity. For example:

    # Download the `xml-file` from the sequencing activity to the current working directory
    download.download_resources(s, workflow, 'sequencing', 'xml-file', output=cwd, progress=tqdm)

### Create batch, pool, sequencing (with QC)

*Complete in web app*

### Demultiplex

In [13]:
activity_label = 'demultiplex'
metadata = {}
resources = {'sample-sheet': [os.path.join(cwd, 'files/sample-sheet.txt')],
            'fastq-file': glob.glob(os.path.join(cwd, "files/*.fastq"), recursive=False),
            'xml-file': [os.path.join(cwd, 'files/demultiplex.xml')]}

In [11]:
demultiplex = workflows.create_activity(s, 
                                        workflow_id, 
                                        activity_label, 
                                        activity=metadata, 
                                        resources=resources, 
                                        progress=tqdm)

AttributeError: 'DataDict' object has no attribute 'uuid'

### Sequencing QC Prep — SortME RNA

In [ ]:
activity_label = 'sequencing-qc-prep-sortmerna'
metadata = {'singleRead': False} # True for paired-end
resources = {'sortmerna-file': [os.path.join(cwd, 'files/sortmerna_single_end.xls')]}

In [ ]:
seq_qc_prep_sortmerna = workflows.create_activity(s, 
                                                  workflow_id, 
                                                  activity_label, 
                                                  data=metadata, 
                                                  resources=resources,
                                                  progress=tqdm)

### Sequencing QC Prep — FastQC

In [ ]:
activity_label = 'sequencing-qc-prep-fastqc'
metadata = {'singleRead': False} # True for paired-end
resources = {'fastqc-file': [os.path.join(cwd, 'files/fastqc_single_end.xls'])}
resource_groups = {'fastqc-output': [os.path.join(cwd, 'files/fastqc')]}

In [ ]:
seq_qc_prep_fastqc = workflows.create_activity(s, 
                                                workflow_id, 
                                                activity_label, 
                                                data=metadata, 
                                                resources=resources,
                                                progress=tqdm)

### Sequencing QC

*Complete in web app*

### Trimmomatic

In [ ]:
activity_label = 'alignment-prep-trimmomatic'
metadata = {}
resources = {'trimmomatic-file': []}

In [ ]:
trimmomatic = workflows.create_activity(s,
                                        workflow_id, 
                                        activity_label, 
                                        data=metadata, 
                                        resources=resources,
                                        progress=tqdm)

### SortME RNA

In [ ]:
activity_label = 'alignment-prep-sortmerna'
metadata = {}
resources = {'sortmerna-file': []}

In [ ]:
trimmomatic = workflows.create_activity(s,
                                        workflow_id, 
                                        activity_label, 
                                        data=metadata, 
                                        resources=resources,
                                        progress=tqdm)

### FastQC

In [ ]:
activity_label = 'alignment-prep-fastqc'
metadata = {}
resources = {'fastqc-file': []}

In [ ]:
fastqc = workflows.create_activity(s,
                                        workflow_id, 
                                        activity_label, 
                                        data=metadata, 
                                        resources=resources,
                                        progress=tqdm)

### Alignment

In [ ]:
activity_label = 'alignment'
metadata = {}
resources = {'bam-file': '/Users/barry/Desktop/Ovation Demo Files/results.txt'}

In [ ]:
alginment = workflows.create_activity(s,
                                      workflow_id, 
                                      activity_label, 
                                      data=metadata, 
                                      resources=resources,
                                      progress=tqdm)

### BAM QC Prep

In [ ]:
activity_label = 'bam-qc-prep'
metadata = {}
resources = {'rna-seqc-file': '/Users/barry/Desktop/Ovation Demo Files/results.txt'}

In [ ]:
bamqc_prep = workflows.create_activity(s,
                                      workflow_id, 
                                      activity_label, 
                                      data=metadata, 
                                      resources=resources,
                                      progress=tqdm)

### BAM QC

*Complete in web app*